In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
class bcolors:
    OK = '\033[92m' #GREEN
    WARNING = '\033[93m' #YELLOW
    FAIL = '\033[91m' #RED
    RESET = '\033[0m' #RESET COLOR

In [3]:
# 改日期
n_day_ago = -2
day = datetime.datetime.strptime(str(datetime.date.today() + datetime.timedelta(days=n_day_ago)) , '%Y-%m-%d' )
day2 = datetime.datetime.strptime(str(datetime.date.today() + datetime.timedelta(days=n_day_ago-7)) , '%Y-%m-%d' )
print(str(day).split(' ')[0])
print(str(day2).split(' ')[0])

2023-06-09
2023-06-02


In [4]:
industry_df = pd.read_excel(r'C:\Users\User\Desktop\code\python\stockdata_VCP\VCP_predict_project\產業別.xlsx').astype(int).astype(str)
rs = pd.read_excel(f'C:/Users/User/Desktop/code/python/stockdata_VCP/VCP_predict_project/history_choosen/{str(day).split(" ")[0]}RS選股.xlsx')['ID'].astype(str)
rs2 = pd.read_excel(f'C:/Users/User/Desktop/code/python/stockdata_VCP/VCP_predict_project/history_choosen/{str(day2).split(" ")[0]}RS選股.xlsx')['ID'].astype(str)

In [5]:
number_of_stock = []
for col in industry_df.columns.values:
    n = len(industry_df.loc[industry_df[col] != '0', col].values)
    number_of_stock.append([col, n])
number_df = pd.DataFrame(number_of_stock, columns=['industry', 'number'])

In [6]:
# number_df = number_df.transpose()
# number_df.to_excel(f'C:/Users/User/Desktop/StockInfoHub/Stock_RS_rate_analysis/100產業分析/100產業RS排行.xlsx')

In [7]:
a = '1808 2457 2634 2727 3032 3078 6691 8222'.split(' ')
all = []
for id in rs:
    stock_ind = []
    for col in industry_df.columns.values:
        if id in industry_df[col].values:
            stock_ind.append(col)
            all.append([col, id, 1])

In [8]:
all_df = pd.DataFrame(all, columns=['industry', 'ID', 'count'])
rs_industry = all_df.groupby(by='industry').sum().sort_values(by='count',ascending=False)
rs_industry['all number'] = 1
for industry in rs_industry.index.values:
    rs_industry.loc[industry, 'all number'] = number_df.loc[number_df['industry']==industry, 'number'].values
    rs_industry.loc[industry, 'percentage'] = round(100*rs_industry.loc[industry, 'count']/rs_industry.loc[industry, 'all number'],1)
rs_industry = rs_industry.transpose()

In [9]:
daily_rs_percentage = pd.DataFrame(rs_industry.loc['percentage'].values,columns=[(day)], index=rs_industry.columns.values).transpose()

In [10]:
a = '1808 2457 2634 2727 3032 3078 6691 8222'.split(' ')
all2 = []
for id in rs2:
    stock_ind = []
    for col in industry_df.columns.values:
        if id in industry_df[col].values:
            stock_ind.append(col)
            all2.append([col, id, 1])

In [11]:
all_df2 = pd.DataFrame(all2, columns=['industry', 'ID', 'count'])
rs_industry2 = all_df2.groupby(by='industry').sum().sort_values(by='count',ascending=False)
rs_industry2['all number'] = 1
for industry in rs_industry2.index.values:
    rs_industry2.loc[industry, 'all number'] = number_df.loc[number_df['industry']==industry, 'number'].values
    rs_industry2.loc[industry, 'percentage'] = round(100*rs_industry2.loc[industry, 'count']/rs_industry2.loc[industry, 'all number'],1)
rs_industry2 = rs_industry2.transpose()

In [12]:
daily_rs_percentage2 = pd.DataFrame(rs_industry2.loc['percentage'].values,columns=[(day2)], index=rs_industry2.columns.values).transpose()

In [13]:
c = pd.concat([daily_rs_percentage, daily_rs_percentage2], axis=1)

In [14]:
# history = pd.read_excel('C:/Users/User/Desktop/StockInfoHub/Stock_RS_rate_analysis/100產業分析/100產業RS排行.xlsx', header=0, index_col=0)

In [15]:
# h = pd.concat([history, daily_rs_percentage2], axis =0)

In [16]:
# h = pd.concat([h, daily_rs_percentage], axis =0)

In [17]:
# h.to_excel('C:/Users/User/Desktop/StockInfoHub/Stock_RS_rate_analysis/100產業分析/100產業RS排行.xlsx')

## py檔測試

In [1]:
import datetime
import requests
import pandas as pd
import numpy as np
import warnings
import datetime
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from email.header import Header
import openpyxl
import traceback
import sys
sys.path.append('C:/Users/User/Desktop/StockInfoHub')
from Shared_Modules.shared_functions import *
from Shared_Modules.shared_variables import *
from stock_data_collector_package.stock_data_collector_utils import *
warnings.filterwarnings('ignore')
class bcolors:
    OK = '\033[92m' #GREEN
    WARNING = '\033[93m' #YELLOW
    FAIL = '\033[91m' #RED
    RESET = '\033[0m' #RESET COLOR
#=============是否寄email============
sentemailornot = True
istest = True
# ============上市股票df============
url = "https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=1&issuetype=1&industry_code=&Page=1&chklike=Y"
response = requests.get(url)
listed = pd.read_html(response.text)[0]
listed.columns = listed.iloc[0,:]
listed = listed[["有價證券代號","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]]
listed = listed.iloc[1:]

# ============上櫃股票df============
urlTWO = "https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=2&issuetype=&industry_code=&Page=1&chklike=Y"
response = requests.get(urlTWO)
listedTWO = pd.read_html(response.text)[0]
listedTWO.columns = listedTWO.iloc[0,:]
listedTWO = listedTWO.loc[listedTWO['有價證券別'] == '股票']
listedTWO = listedTWO[["有價證券代號","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]]

# ============上市股票代號+.TW============
stock_1 = listed["有價證券代號"]
stock_num = stock_1.apply(lambda x: str(x) + ".TW")
stock_num.loc[len(stock_num)+1] = '0050.TW'
stock_num.loc[len(stock_num)+1] = '^TWII'
# print(stock_num)

# ============上櫃股票代號+.TWO============
stock_2 = listedTWO["有價證券代號"]
stock_num2 = stock_2.apply(lambda x: str(x) + ".TWO")
# print(stock_num2)

# ============concate全部股票代號============
stock_num = pd.concat([stock_num, stock_num2], ignore_index=True)
# print(stock_num)
allstock_info = pd.concat([listed, listedTWO], ignore_index=True)
allstock_info.columns = ["ID","有價證券名稱","市場別","產業別","公開發行/上市(櫃)/發行日"]
allstock_info.set_index('ID', inplace = True)
print(allstock_info)
# print(type(allstock_info['ID'].values[0]))

#============ 把要看日期的個股資料合併(一天)============
# for i in tqdm(range(0,300,1), postfix='*******', ncols=150):
for i in [0]:

# ============改日期============
    # 今天日期往前推幾天(今日 - n_day_ago)
    n_day_ago = 0
    try:
        day = datetime.datetime.strptime(str(datetime.date.today() - datetime.timedelta(days=n_day_ago)) , '%Y-%m-%d' )
        if str(day).split(' ')[0] in HOLIDAY:
            line_notify(f'{day}放假不執行daily_rs_industry.py', TOKEN_FOR_UPDATE)
            sys.exit()
        print('day: ', day)
        line_notify(f'{day}開始執行daily_rs_industry.py', TOKEN_FOR_UPDATE)
        ## ============第一個日期選出符合條件的股票並匯出excel============
        allstock = concat_stock(day, stock_num[2:])
        line_notify('concat_stock OK', TOKEN_FOR_UPDATE)
        # fail_ID = ['1101.TW', '1102.TW', '1103.TW', '1104.TW', '1108.TW', '1109.TW', '1201.TW', '1203.TW', '1213.TW', '1229.TW', '1234.TW', '1313.TW', '1435.TW', '1733.TW', '2434.TW', '2440.TW', '2528.TW', '4569.TW', '6225.TW', '6655.TW', '2740.TWO', '2752.TWO', '3085.TWO', '3226.TWO', '3332.TWO', '3426.TWO', '3523.TWO', '4192.TWO', '4556.TWO', '4568.TWO', '4712.TWO', '4806.TWO', '5016.TWO', '5455.TWO', '5520.TWO', '5601.TWO', '6103.TWO', '6219.TWO', '6236.TWO', '6629.TWO', '8077.TWO', '8342.TWO', '8423.TWO', '8472.TWO', '8917.TWO', '8921.TWO', '9960.TWO']
        # fail_ID = list(map(lambda x:x.split('.')[0], fail_ID))
        # for failid in fail_ID:
        #     try:
        #         allstock.drop(failid, inplace=True)
        #     except:
                # pass

        # ============加入是否符合策略============
        allstock = template(allstock,allstock_info)
        line_notify('template OK', TOKEN_FOR_UPDATE)
        # ID = allstock.index.values
        allstock.dropna(inplace=True)
        print(allstock)
        T5_ID = allstock.loc[allstock['T5']].index.values.astype(str)
        T5_2_ID = allstock.loc[allstock['T5-2']].index.values.astype(str)
        T6_ID = allstock.loc[allstock['T6'], ['產業別', 'RS EMA250rate']].sort_values(by='RS EMA250rate', ascending=False).iloc[:50].index.values.astype(str)
        T11_ID = allstock.loc[allstock['T11']].index.values.astype(str)
        TM_ID = allstock.loc[allstock['TM']].index.values.astype(str)
        
        T5_stock_num = len(T5_ID)
        T5_2_stock_num = len(T5_2_ID)
        T6_stock_num = len(T6_ID)
        T11_stock_num = len(T11_ID)
        TM_stock_num = len(TM_ID)
        all_template_ID = list(set(np.concatenate([T5_ID,T5_2_ID,T6_ID,T11_ID,TM_ID])))
        all_template_ID_exTM = list(set(np.concatenate([T5_ID,T5_2_ID,T6_ID,T11_ID])))
        print('T5 股票數量 : ', T5_stock_num, end=' | ')
        print('T5-2 股票數量 : ', T5_2_stock_num, end=' | ')
        print('T6 股票數量 : ', T6_stock_num, end=' | ')
        print('T11 股票數量 : ', T11_stock_num, end=' | ')
        print('TM 股票數量 : ', TM_stock_num)
        print('除了TM以外的股票數量 : ', len(all_template_ID))
        print('全部股票數量 : ', len(all_template_ID_exTM))
        
        # ============防止用濾網篩選前50個的template混亂============
        allstock[['T5','T5-2','T6','T11','TM']] = False
        allstock.loc[T5_ID,'T5'] = True
        allstock.loc[T5_2_ID,'T5-2'] = True
        allstock.loc[T6_ID,'T6'] = True
        allstock.loc[T11_ID,'T11'] = True
        allstock.loc[TM_ID,'TM'] = True
        allstock[['RS 250rate', 'RS 50rate', 'RS 20rate', 'RS EMA250rate', 'RS EMA50rate', 'RS EMA20rate']] = allstock[['RS 250rate', 'RS 50rate', 'RS 20rate', 'RS EMA250rate', 'RS EMA50rate', 'RS EMA20rate']].astype(float).round(1)
        allstock['Volume 50MA'] = (allstock['Volume 50MA']/1000).astype(int)
        # for i,j in enumerate(allstock['TM']):
        #     if j != j:
        #         print(j, i, ID[i])
        #         allstock.drop(ID[i], axis=0, inplace=True)
        print(f'{bcolors.OK}All Template Stocks... {bcolors.RESET}', end='')

        #============今日符合策略全部股票============
        apexstock = allstock.loc[all_template_ID, ['Name', 'Adj Close', 'busness volume(億)', 'Volume 50MA', 'business volume 50MA(百萬)', '產業別'
                                            , 'RS EMA250rate', 'RS EMA50rate', 'RS EMA20rate', 'RS 250rate', 'RS 50rate', 'RS 20rate'
                                            , 'ATR250/price', 'ATR50/price', 'ATR20/price', 'STD7', 'STD20', 'STD50'
                                            ,'RS 250EMA is 10MAX','RS 50EMA is 10MAX','RS 20EMA is 10MAX', 'T5', 'T5-2', 'T6', 'T11', 'TM'
                                            , 'Price>150MA', 'Price>200MA', '50MA>150MA', '50MA>200MA', '150MA>200MA','year high sort', 'year low sort'
                                            , '200MA trending up 60d', 'Volume 50MA>100k', 'Volume 50MA>250k']]
        
        column_name = ['Name', 'Adj Close', 'busness volume(億)', 'Volume 50MA(張)', 'business volume 50MA(百萬)', '產業別'
                , 'ES250rate', 'ES50rate', 'ES20rate', 'S250rate', 'S50rate', 'S20rate'
                , 'ATR250/price', 'ATR50/price', 'ATR20/price', 'STD7', 'STD20', 'STD50'
                ,'ES250 is 10D MAX','ES50 is 10D MAX','ES20 is 10D MAX', 'T5', 'T5-2', 'T6', 'T11', 'TM'
                , 'Price>150MA', 'Price>200MA', '50MA>150MA', '50MA>200MA', '150MA>200MA','year high sort', 'year low sort'
                , '200MA trending up 60d', 'Volume 50MA>100k', 'Volume 50MA>250k']

        apexstock.columns = column_name
        apexstock = apexstock.dropna()
        print(f'{bcolors.OK}OK{bcolors.RESET}')
        line_notify('all template stocks OK', TOKEN_FOR_UPDATE)
        print(f'All Template Stocks shape:{apexstock.shape}')
        
        print(f'{bcolors.OK}RS EMA250rate>80 Stocks... {bcolors.RESET}', end='')

        #============今日策略250ERS>80股票============
        apexstock2 = allstock.loc[allstock['RS EMA250rate>80'], ['Name', 'Adj Close', 'busness volume(億)', 'Volume 50MA', 'business volume 50MA(百萬)', '產業別'
                                            , 'RS EMA250rate', 'RS EMA50rate', 'RS EMA20rate', 'RS 250rate', 'RS 50rate', 'RS 20rate'
                                            , 'ATR250/price', 'ATR50/price', 'ATR20/price', 'STD7', 'STD20', 'STD50'
                                            ,'RS 250EMA is 10MAX','RS 50EMA is 10MAX','RS 20EMA is 10MAX', 'T5', 'T5-2', 'T6', 'T11', 'TM'
                                            , 'Price>150MA', 'Price>200MA', '50MA>150MA', '50MA>200MA', '150MA>200MA','year high sort', 'year low sort'
                                            , '200MA trending up 60d', 'Volume 50MA>100k', 'Volume 50MA>250k']]
        # apexstock2[['RS 250rate', 'RS 50rate', 'RS 20rate','RS EMA250rate', 'RS EMA50rate', 'RS EMA20rate']] = apexstock2[['RS 250rate', 'RS 50rate', 'RS 20rate','RS EMA250rate', 'RS EMA50rate', 'RS EMA20rate']].astype(float).round(0)
        apexstock2.columns = column_name
        apexstock2 = apexstock2.dropna()
        print(f'{bcolors.OK}OK{bcolors.RESET}')
        line_notify('RS EMA 250rate>80 stocks OK', TOKEN_FOR_UPDATE)
        print(f'RS EMA250rate>80 Stocks shape:{apexstock2.shape}')

        print(f'{bcolors.OK}All Stocks... {bcolors.RESET}', end='')

        #============今日全部股票============
        sentstock = allstock.copy()
        allstock = allstock.loc[:, ['Name', 'Adj Close', 'busness volume(億)', 'Volume 50MA', 'business volume 50MA(百萬)', '產業別'
                                            , 'RS EMA250rate', 'RS EMA50rate', 'RS EMA20rate', 'RS 250rate', 'RS 50rate', 'RS 20rate'
                                            , 'ATR250/price', 'ATR50/price', 'ATR20/price', 'STD7', 'STD20', 'STD50'
                                            ,'RS 250EMA is 10MAX','RS 50EMA is 10MAX','RS 20EMA is 10MAX', 'T5', 'T5-2', 'T6', 'T11', 'TM'
                                            , 'Price>150MA', 'Price>200MA', '50MA>150MA', '50MA>200MA', '150MA>200MA','year high sort', 'year low sort'
                                            , '200MA trending up 60d', 'Volume 50MA>100k', 'Volume 50MA>250k']]
        # allstock[['RS 250rate', 'RS 50rate', 'RS 20rate','RS EMA250rate', 'RS EMA50rate', 'RS EMA20rate']] = allstock[['RS 250rate', 'RS 50rate', 'RS 20rate','RS EMA250rate', 'RS EMA50rate', 'RS EMA20rate']].astype(float).round(0)
        allstock.columns = column_name
        allstock = allstock.dropna()
        print(f'{bcolors.OK}OK{bcolors.RESET}')
        line_notify('all stocks OK', TOKEN_FOR_UPDATE)
        print(f'All Stocks Shape:{allstock.shape}{bcolors.RESET}')
        
        # print(len(apexstock.index))

        #============匯出全個股條件篩選excel============
        sent_file_path = 'C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/全個股條件篩選/' + str(day).split(' ')[0] + '選股' + '.xlsx'
        allstock.to_excel(sent_file_path, encoding='utf-8-sig')

        #============匯出符合策略股票excel============
        apexstock.to_excel('C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/250RS選股/' + str(day).split(' ')[0] + '250RS選股' + '.xlsx', encoding='utf-8-sig')

        #============匯出250ERS>80股票excel============
        apexstock2.to_excel('C:/Users/User/Desktop/StockInfoHub/Stock_Data_Collector/250RS選股/' + str(day).split(' ')[0] + 'EMA250RS選股' + '.xlsx', encoding='utf-8-sig')
        try:
            rs_industry(day)
        except:
            line_notify('⚠️rs_industry FAIL', TOKEN_FOR_UPDATE)
            sys.exit()
        line_notify('rs_industry OK', TOKEN_FOR_UPDATE)
        try:
            top_businessvolume_industry(day)
        except:
            line_notify('⚠️top_volume_industry FAIL', TOKEN_FOR_UPDATE)
            sys.exit()
        line_notify('top_volume_industry OK', TOKEN_FOR_UPDATE)
        print(str(day))
        ###############################################################################################################################################################################
        ##                                                                                寄email                                                                                    ##
        ############################################################################################################################################################################### 
        
        if sentemailornot == True:
            ers_higher80 = sentstock.loc[sentstock['RS EMA250rate>80']]
            ers_higher80_length = len(ers_higher80.index)
            all_length = len(sentstock.index)
            numb_Price_higher_20MA = len(ers_higher80.loc[ers_higher80['Price>20MA'], 'Price>20MA'].index.values)
            numb_Price_higher_50MA = len(ers_higher80.loc[ers_higher80['Price>50MA'], 'Price>50MA'].index.values)
            numb_Price_higher_150MA = len(ers_higher80.loc[ers_higher80['Price>150MA'], 'Price>150MA'].index.values)
            numb_Price_higher_200MA = len(ers_higher80.loc[ers_higher80['Price>200MA'], 'Price>200MA'].index.values)
            all_numb_Price_higher_20MA = len(sentstock.loc[sentstock['Price>20MA'], 'Price>20MA'].index.values)
            all_numb_Price_higher_50MA = len(sentstock.loc[sentstock['Price>50MA'], 'Price>50MA'].index.values)
            all_numb_Price_higher_150MA = len(sentstock.loc[sentstock['Price>150MA'], 'Price>150MA'].index.values)
            all_numb_Price_higher_200MA = len(sentstock.loc[sentstock['Price>200MA'], 'Price>200MA'].index.values)
            print(f'{bcolors.OK}寄送email...{bcolors.RESET}')
            

            ## 比較今天的選股跟昨天的選股

            industry_df = pd.read_excel(r'C:\Users\User\Desktop\StockInfoHub\others\產業別.xlsx').astype(int).astype(str)
            number_of_stock_in_industry = pd.read_excel(r'C:\Users\User\Desktop\StockInfoHub\Stock_RS_rate_analysis\100產業分析\100產業RS排行.xlsx', index_col=0, header=0).loc['number'].astype(int)
            industry_category_df = []
            
            # ============讀今日和昨日的股票Excel============
            path = r'C:\Users\User\Desktop\StockInfoHub\Stock_Data_Collector\全個股條件篩選\\'
            new_date = str(day).split(' ')[0]
            for delay_day in range(1, 7):
                old_date = str(datetime.datetime.strptime(str(datetime.date.today() - datetime.timedelta(days=n_day_ago+delay_day)) , '%Y-%m-%d' )).split(' ')[0]
                print(f'New date:{new_date} | Old date:{old_date}')
                try:
                    df_old = pd.read_excel(f'{path}{old_date}選股.xlsx')
                    df_new = pd.read_excel(f'{path}{new_date}選股.xlsx')
                    df_old.set_index('ID', inplace=True)
                    df_new.set_index('ID', inplace=True)
                    break
                except:
                    continue            
            print(f'New date:{new_date} | Old date:{old_date}')

            # ============今日和昨日股票的ID============
            df_same_ID = df_new.loc[df_new.index.isin(df_old.index.values)].index.values
            df_old = df_old.loc[df_same_ID]
            df_new = df_new.loc[df_same_ID]
            old_T5_ID = df_old.loc[df_old['T5']].index.values
            old_T5_2_ID = df_old.loc[df_old['T5-2']].index.values
            old_T6_ID = df_old.loc[df_old['T6']].index.values
            old_T11_ID = df_old.loc[df_old['T11']].index.values
            new_T5_ID = df_new.loc[df_new['T5']].index.values
            new_T5_2_ID = df_new.loc[df_new['T5-2']].index.values
            new_T6_ID = df_new.loc[df_new['T6']].index.values
            new_T11_ID = df_new.loc[df_new['T11']].index.values

            # ============新增和刪除股票的ID============
            T5_add_ID = np.sort([i for i in new_T5_ID if i not in old_T5_ID])
            T5_drop_ID = np.sort([i for i in old_T5_ID if i not in new_T5_ID])
            T6_add_ID = np.sort([i for i in new_T6_ID if i not in old_T6_ID])
            T6_drop_ID = np.sort([i for i in old_T6_ID if i not in new_T6_ID])
            T11_add_ID = np.sort([i for i in new_T11_ID if i not in old_T11_ID])
            T11_drop_ID = np.sort([i for i in old_T11_ID if i not in new_T11_ID])
            T5_2_add_ID = np.sort([i for i in new_T5_2_ID if i not in old_T5_2_ID])
            T5_2_drop_ID = np.sort([i for i in old_T5_2_ID if i not in new_T5_2_ID])
            T5_drop_goodID = np.sort([i for i in T5_drop_ID if all([df_new.loc[i,'S250rate']>=75, df_new.loc[i, 'S20rate']>85])])
            T5_2_drop_goodID = np.sort([i for i in T5_2_drop_ID if all([df_new.loc[i,'ES250rate']>=75, df_new.loc[i, 'ES20rate']>85])])

            # ============email新增股票數量的文字============
            stock_numbers_all_template_text = f'<pre>策略股票數量 : \n\n\
T5(new):{len(new_T5_ID)} | T5_2(new):{len(new_T5_2_ID)} | T6(new):{len(new_T6_ID)} | T11(new):{len(new_T11_ID)} | all(new):{len(set(np.concatenate([new_T5_ID, new_T5_2_ID, new_T6_ID, new_T11_ID])))}\n\n\
T5(old):{len(old_T5_ID)} | T5_2(old):{len(old_T5_2_ID)} | T6(old):{len(old_T6_ID)} | T11(old):{len(old_T11_ID)} | all(old):{len(set(np.concatenate([old_T5_ID, old_T5_2_ID, old_T6_ID, old_T11_ID])))}\n\n\
T5(add):{len(T5_add_ID)} | T5_2(add):{len(T5_2_add_ID)} | T6(add):{len(T6_add_ID)} | T11(add):{len(T11_add_ID)} | all(add):{len(set(np.concatenate([T5_add_ID, T5_2_add_ID, T6_add_ID, T11_add_ID])))}\n\n\
T5(drop):{len(T5_drop_ID)} | T5_2(drop):{len(T5_2_drop_ID)} | T6(drop):{len(T6_drop_ID)} | T11(drop):{len(T11_drop_ID)} | all(drop):{len(set(np.concatenate([T5_drop_ID, T5_2_drop_ID, T6_drop_ID, T11_drop_ID])))}\n\n\</pre>'
            # print add and drop stock ID
            all_add_ID = np.array(list(set(np.concatenate([T5_add_ID, T5_2_add_ID, T6_add_ID, T11_add_ID]))))
            all_drop_ID = np.array(list(set(np.concatenate([T5_drop_ID, T5_2_drop_ID, T6_drop_ID]))))

            # ============email新增股票ID的文字============
            add_stockID_text = f'<pre>\n策略股票新增ID : \nT5新增股票ID : \n{T5_add_ID}\n\nT5RS破高刪除的股票ID : \n{T5_drop_goodID}\n\nT5_2新增股票ID : \n{T5_2_add_ID}\n\nT5_2RS破高刪除的股票ID : \n{T5_2_drop_goodID}\n\n\
T6新增股票ID : \n{T6_add_ID}\n\nT11新增股票ID : \n{T11_add_ID}\n</pre>'
            add_industry_df = print_industry_category_df(industry_df,all_add_ID,[],number_of_stock_in_industry, t='新增')
            drop_industry_df = print_industry_category_df(industry_df,all_drop_ID,[],number_of_stock_in_industry, t='刪除')
            all_industry_df = pd.concat([add_industry_df, drop_industry_df], axis=0)
            all_industry_df.fillna(0, inplace=True)
            all_industry_df.loc['合計新增數量'] = all_industry_df.loc['該類股新增數量'] - all_industry_df.loc['該類股刪除數量']
            all_industry_df.loc['合計新增股票%數'] = all_industry_df.loc['新增股票%數'] - all_industry_df.loc['刪除股票%數']
            all_industry_df = all_industry_df.transpose()
            all_industry_df.sort_values(by='合計新增股票%數', ascending=False, inplace=True)
            # all_industry_df['新增股票數量排名'] = all_industry_df['該類股新增數量'].rank(ascending=False)
            display(all_industry_df)
            add_industry_df = all_industry_df[['該類股新增數量', '該類股刪除數量', '合計新增數量', '合計新增股票%數']].iloc[:30]
            add_industry_df['合計新增數量排名'] = add_industry_df['合計新增數量'].rank(ascending=False)
            add_industry_df['該類股總數量'] = number_of_stock_in_industry.loc[add_industry_df.index.values]
            add_industry_df = add_industry_df.loc[add_industry_df['合計新增數量']>0]
            add_industry_df.sort_values(by='合計新增股票%數', ascending=False, inplace=True)
            add_industry_df = add_industry_df.iloc[:, [5,0,1,2,3,4]].transpose()
            add_industry_df.iloc[[0,1,2,3,5]] = add_industry_df.iloc[[0,1,2,3,5]].astype(int)
            drop_industry_df = all_industry_df[['該類股新增數量', '該類股刪除數量', '合計新增數量', '合計新增股票%數']].iloc[-30:]
            drop_industry_df['合計刪除數量排名'] = drop_industry_df['合計新增數量'].rank(ascending=True)
            drop_industry_df['該類股總數量'] = number_of_stock_in_industry.loc[drop_industry_df.index.values]
            drop_industry_df = drop_industry_df.loc[drop_industry_df['合計新增數量']<0]
            drop_industry_df.sort_values(by='合計新增股票%數', ascending=True, inplace=True)
            drop_industry_df = drop_industry_df.iloc[:, [5,0,1,2,3,4]].transpose()
            drop_industry_df.iloc[[0,1,2,3,5]] = drop_industry_df.iloc[[0,1,2,3,5]].astype(int)
            # ============email新增股票產業的表格文字============
            html_addstock_table = add_industry_df.to_html(index=True, header=True, justify='center')
            html_dropstock_table = drop_industry_df.to_html(index=True, header=True, justify='center')

            # ============email股票位階的文字============
            price_position_text = f'<pre>{new_date} ERS>80股票位階 : \n\
{numb_Price_higher_20MA}檔({str(round(100*numb_Price_higher_20MA/ers_higher80_length,2))}%)股票的收盤價大於20MA\n\
{numb_Price_higher_50MA}檔({str(round(100*numb_Price_higher_50MA/ers_higher80_length,2))}%)股票的收盤價大於50MA\n\
{numb_Price_higher_150MA}檔({str(round(100*numb_Price_higher_150MA/ers_higher80_length,2))}%)股票的收盤價大於150MA\n\
{numb_Price_higher_200MA}檔({str(round(100*numb_Price_higher_200MA/ers_higher80_length,2))}%)股票的收盤價大於200MA\n\n\
{new_date} 全部股票位階 : \n\
{all_numb_Price_higher_20MA}檔({str(round(100*all_numb_Price_higher_20MA/all_length,2))}%)股票的收盤價大於20MA\n\
{all_numb_Price_higher_50MA}檔({str(round(100*all_numb_Price_higher_50MA/all_length,2))}%)股票的收盤價大於50MA\n\
{all_numb_Price_higher_150MA}檔({str(round(100*all_numb_Price_higher_150MA/all_length,2))}%)股票的收盤價大於150MA\n\
{all_numb_Price_higher_200MA}檔({str(round(100*all_numb_Price_higher_200MA/all_length,2))}%)股票的收盤價大於200MA\n</pre>'
            
            # ============email全部文字============
            message_text = f'{new_date} vs {old_date}每日選股\n\n\
{stock_numbers_all_template_text}\n\
{add_stockID_text}\n\n'
            
            # ============產業類股ERS>80增減最多============
            ERS100 = pd.read_excel('C:/Users/User/Desktop/StockInfoHub/Stock_RS_rate_analysis/100產業分析/100產業RS排行.xlsx', header=0, index_col=0)
            yesterdat = str(datetime.datetime.strptime(str(datetime.date.today() - datetime.timedelta(days=n_day_ago+delay_day)) , '%Y-%m-%d' ))
            for i in range(7):
                lastweek = str(datetime.datetime.strptime(str(datetime.date.today() - datetime.timedelta(days=n_day_ago+7+i)) , '%Y-%m-%d' ))
                try:
                    ERS100.loc[str(lastweek)]
                    print(f'lastweek: {lastweek}')
                    break
                except:
                    continue
            new_rs = ERS100.loc[str(day)]
            old_rs = ERS100.loc[str(yesterdat)]
            lastweek_rs = ERS100.loc[str(lastweek)]
            add = new_rs - old_rs
            add.fillna(0, inplace=True)
            add = pd.concat([add, ERS100.loc['number']], axis=1)
            add.columns = ['add %', 'Num of stock in industry']
            add.sort_values(by = 'add %', ascending=False, inplace=True)
            first15_add = add.iloc[:15].transpose().to_html(index=True, header=True, justify='center')
            last15_add = add.iloc[-15:].transpose().iloc[:, ::-1].to_html(index=True, header=True, justify='center')
            lastweek_add = new_rs - lastweek_rs
            lastweek_add.fillna(0, inplace=True)
            lastweek_add = pd.concat([lastweek_add, ERS100.loc['number']], axis=1)
            lastweek_add.columns = ['weekly add %', 'Num of stock in industry']
            lastweek_add.sort_values(by = 'weekly add %', ascending=False, inplace=True)
            lastweek_first15_add = lastweek_add.iloc[:15].transpose().to_html(index=True, header=True, justify='center')
            lastweek_last15_add = lastweek_add.iloc[-15:].transpose().iloc[:, ::-1].to_html(index=True, header=True, justify='center')
            lastweek_add['count'] = lastweek_add['weekly add %']*lastweek_add['Num of stock in industry']/100
            lastweek_add.sort_values(by = 'count', ascending=False, inplace=True)
            lastweek_first15_add_count = lastweek_add.iloc[:15].drop('weekly add %', axis = 1).transpose().astype(int).to_html(index=True, header=True, justify='center')
            lastweek_last15_add_count = lastweek_add.iloc[-15:].drop('weekly add %', axis = 1).transpose().astype(int).iloc[:, ::-1].to_html(index=True, header=True, justify='center')
            message = """
            <html>
            <body>
                <p>{}</p>
                <p>{}</p>
                <p>新增策略股票產業別</p>
                {}
                <p>刪除策略股票產業別</p>
                {}
                <p>產業類股ERS>80今日增減%最多</p>
                {}
                {}
                <p>產業類股ERS>80一周內增減%最多</p>
                {}
                {}
                <p>產業類股ERS>80一周內增減數量最多</p>
                {}
                {}
            </body>
            </html>
            """.format(price_position_text, message_text, html_addstock_table, html_dropstock_table, first15_add, last15_add, lastweek_first15_add, lastweek_last15_add, lastweek_first15_add_count, lastweek_last15_add_count)
            #                <p>產業類股ERS>80一周內減少數量最多</p>
            # ============邮件服务器的信息 - 使用Gmail的SMTP服务器============
            smtp_server = 'smtp.gmail.com'
            smtp_port = 587  # Gmail的TLS端口号
            password = 'uqkz xwdm apft zmpr'  # 发件人的Gmail密码或应用程序专用密码
            
            # ============发件人和收件人信息============
            sender_email = 'johnnn1231232@gmail.com'  # 发件人的Gmail电子邮件地址
            if istest:
                receiver_email = ['johnnystockinfo@gmail.com']
            else:
                receiver_email = ['johnnystockinfo@gmail.com', 'jack12211221@gmail.com', 'kaiwenyang708@gmail.com']  # 收件人的Gmail电子邮件地址, 'jack12211221@gmail.com', 'kaiwenyang708@gmail.com', 'youping0116@gmail.com'
            
            # ============创建邮件对象============
            msg = MIMEMultipart()
            msg['From'] = sender_email
            msg['To'] = ', '.join(receiver_email)
            if istest:
                msg['Subject'] = Header(f"{str(day).split(' ')[0]}-測試-每日選股", 'utf-8').encode()
            else:
                msg['Subject'] = Header(f"{str(day).split(' ')[0]}每日選股", 'utf-8').encode()
            msg.attach(MIMEText(message, 'html'))

            # ============打开要附加的xlsx文件============
            workbook = openpyxl.load_workbook(f'{path}{new_date}選股.xlsx')

            # ============保存工作簿为.xlsx文件============
            xlsx_filename = f'{new_date}選股.xlsx'
            workbook.save(xlsx_filename)

            # ============将.xlsx文件附加到邮件============
            with open(xlsx_filename, 'rb') as xlsx_file:
                excel_attachment = MIMEApplication(xlsx_file.read(), _subtype='xlsx')
                excel_attachment.add_header('Content-Disposition', 'attachment', filename=Header(xlsx_filename, 'utf-8').encode())
                msg.attach(excel_attachment)
            
            # ============连接到SMTP服务器============
            try:
                server = smtplib.SMTP('smtp.gmail.com', 587)
                server.starttls()
                server.login(sender_email, password)
                
                # ============发送邮件============
                server.sendmail(sender_email, receiver_email, msg.as_string())
                print(f'{bcolors.OK}寄送email OK{bcolors.RESET}')
                line_notify('sent email OK', TOKEN_FOR_UPDATE)
            except Exception as e:
                traceback.print_exc()
                print(f'\n{bcolors.FAIL}{e}{bcolors.RESET}')
                line_notify('⚠️sent email FAIL', TOKEN_FOR_UPDATE)
            finally:
                # ============关闭SMTP连接============
                server.quit()
            # ============删除附加的.xlsx文件============
            os.remove(xlsx_filename)
    except Exception as e:
        traceback.print_exc()
        print(f'\n{bcolors.FAIL}{e}{bcolors.RESET}')
        pass
line_notify(f'{day}daily_rs_industry.py執行完成', TOKEN_FOR_UPDATE)

Shared module initialized.
['2023-10-09', '2023-10-10', '2024-01-01', '2024-02-06', '2024-02-07', '2024-02-08', '2024-02-09', '2024-02-10', '2024-02-11', '2024-02-12', '2024-02-13', '2024-02-14', '2024-02-28', '2024-04-04', '2024-04-05', '2024-05-01', '2024-06-10', '2024-09-17', '2024-10-10']
     有價證券名稱 市場別    產業別 公開發行/上市(櫃)/發行日
ID                                   
1101     台泥  上市   水泥工業     1962/02/09
1102     亞泥  上市   水泥工業     1962/06/08
1103     嘉泥  上市   水泥工業     1969/11/14
1104     環泥  上市   水泥工業     1971/02/01
1108     幸福  上市   水泥工業     1990/06/06
...     ...  ..    ...            ...
9949     琉園  上櫃  文化創意業     2003/11/21
9950    萬國通  上櫃   塑膠工業     2004/02/17
9951     皇田  上櫃   電機機械     2003/12/18
9960    邁達康  上櫃   運動休閒     2004/12/06
9962     有益  上櫃   鋼鐵工業     2006/07/10

[1804 rows x 4 columns]
day:  2024-01-18 00:00:00
Stocks Concatenation...
            Open  High    Low Close Adj Close    Volume    ID    5MA   10MA  \
2024-01-18  18.0  18.1  17.85  18.1      18.1  166119.0  1

,該類股新增數量,該類股總數量,新增股票%數,新增股票數量排名,該類股刪除數量,該類股總數量,刪除股票%數,刪除股票數量排名,合計新增數量,合計新增股票%數
光碟片,1.0,3.0,33.33,15.0,0.0,0.0,0.00,0.0,1.0,33.33
Internet技術與基礎設施,2.0,17.0,11.76,6.0,0.0,0.0,0.00,0.0,2.0,11.76
IC封裝測試,3.0,24.0,12.50,3.0,1.0,24.0,4.17,23.0,2.0,8.33
運動產業,1.0,15.0,6.67,15.0,0.0,0.0,0.00,0.0,1.0,6.67
軟體業,4.0,67.0,5.97,2.0,0.0,0.0,0.00,0.0,4.0,5.97
休閒娛樂,1.0,17.0,5.88,15.0,0.0,0.0,0.00,0.0,1.0,5.88
遊戲產業,1.0,22.0,4.55,15.0,0.0,0.0,0.00,0.0,1.0,4.55
家電,2.0,25.0,8.00,6.0,1.0,25.0,4.00,23.0,1.0,4.00
太陽能,1.0,25.0,4.00,15.0,0.0,0.0,0.00,0.0,1.0,4.00
金融業,1.0,46.0,2.17,15.0,0.0,0.0,0.00,0.0,1.0,2.17


lastweek: 2024-01-11 00:00:00
寄送email OK
